In [ ]:
install.packages('glmnet')

In [ ]:
library('MASS')

In [ ]:
N = 250
P = 500

In [ ]:
K = floor(P/3)
alpha = array(rnorm(K*P),c(K,P))
W = array(rnorm(N*K),c(N,K))
W_test = array(rnorm(N*10*K),c(N*10,K))

In [ ]:
X = W%*%alpha

In [ ]:
X_test = W_test%*%alpha

In [ ]:
dim(X_test)

In [ ]:
beta = rnorm(P)
beta[50:500] <- 0 

In [ ]:
plot(beta)

In [ ]:
sig = P*.05

In [ ]:
y = X%*%beta + rnorm(N,sd=sig)

In [ ]:
plot(y,X%*%beta)

In [ ]:
y_test = X_test%*%beta + rnorm(N*10,sd=sig)

In [ ]:
lin_mod = lm(y~1+X)

In [ ]:
tail(lin_mod$coef)

In [ ]:
lambda = exp(seq(-20,20,length.out=100))
lambda

In [ ]:
install.packages('glmnet')

In [ ]:
library('glmnet')

In [ ]:
fit.ridge = glmnet(x=X,y=y,family="gaussian",alpha=0,lambda=lambda)

In [ ]:
dim(fit.ridge$beta)

In [ ]:
matplot(log(fit.ridge$lambda),t(fit.ridge$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
cv.ridge = cv.glmnet(x=X,y=y,family="gaussian",alpha=0,lambda=lambda)

In [ ]:
plot(cv.ridge)

In [ ]:
lam_hat = cv.ridge$lambda.min
log(lam_hat)

In [ ]:
lam_hat = cv.ridge$lambda.1se
log(lam_hat)

In [ ]:
best.ridge = glmnet(x=X,y=y,family="gaussian",alpha=0,lambda=lam_hat)

In [ ]:
names(best.ridge)

In [ ]:
best.ridge$a0

In [ ]:
plot(beta,as.vector(best.ridge$beta))

In [ ]:
plot(y_test,predict(best.ridge,newx=X_test))
abline(coef=c(0,1),col='red')

In [ ]:
sqrt(mean((y_test-predict(best.ridge,newx=X_test))^2))

### lasso 

In [ ]:
fit.lasso = glmnet(x=X,y=y,family="gaussian",alpha=1,lambda=lambda)

In [ ]:
matplot(log(fit.lasso$lambda),t(fit.lasso$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
cv.lasso = cv.glmnet(x=X,y=y,family="gaussian",alpha=1,lambda=lambda)

In [ ]:
plot(cv.lasso)

In [ ]:
lambda = exp(seq(-5,5,length.out=100))

In [ ]:
cv.lasso = cv.glmnet(x=X,y=y,family="gaussian",alpha=1,lambda=lambda)

In [ ]:
plot(cv.lasso)

In [ ]:
lam_hat = cv.lasso$lambda.min
log(lam_hat)

In [ ]:
lam_hat = cv.lasso$lambda.1se
log(lam_hat)

In [ ]:
best.lasso = glmnet(x=X,y=y,family="gaussian",alpha=1,lambda=lam_hat)

In [ ]:
plot(beta,as.vector(best.lasso$beta))

In [ ]:
plot(y_test,predict(best.lasso,newx=X_test));
abline(coef=c(0,1),col='red')

In [ ]:
sqrt(mean((y_test-predict(best.ridge,newx=X_test))^2))

In [ ]:
sqrt(mean((y_test-predict(best.lasso,newx=X_test))^2))

In [ ]:
mean(abs(best.lasso$beta)>0)

### double lasso / relaxed lasso

In [ ]:
sel = which(abs(best.lasso$beta)>0)

In [ ]:
length(sel)

In [ ]:
X_sel = X[,sel]

In [ ]:
lam2 = exp(seq(-10,10,length.out=100))
dlasso.cv = cv.glmnet(x=X_sel,y=y,family="gaussian",alpha=1,lambda=lam2)

In [ ]:
plot(dlasso.cv)

In [ ]:
lambd = dlasso.cv$lambda.1se

In [ ]:
dlasso = glmnet(x=X_sel,y=y,family="gaussian",alpha=1,lambda=lambd)

In [ ]:
preds = predict(dlasso,newx=X_test[,sel])
plot(y_test,preds)
abline(coef=c(0,1),col='red')

In [ ]:
sqrt(mean((y_test-preds)^2))

### elastic net

In [ ]:
lam_seq = exp(seq(-10,10,length.out=100))
fit.en = glmnet(x=X,y=y,family="gaussian",alpha=1/2,lambda=lam_seq)
fit.en.cv = cv.glmnet(x=X,y=y,family="gaussian",alpha=1/2,lambda=lam_seq)

In [ ]:
matplot(log(fit.en$lambda),t(fit.en$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
plot(fit.en.cv)

In [ ]:
fit.en = glmnet(x=X,y=y,family="gaussian",alpha=1/2,lambda=fit.en.cv$lambda.1se)

In [ ]:
preds = predict(fit.en,newx=X_test)
plot(y_test,preds)
abline(coef=c(0,1),col='red')

In [ ]:
sqrt(mean((y_test-preds)^2))